## 1 - Download data

In [3]:
from openimages.download import download_dataset
download_dataset("./../data/", ["Traffic light"], 'exclusions.txt', limit=200, annotation_format='pascal')

2025-04-29  08:27:09 INFO Downloading 200 train images for class 'traffic light'
100%|██████████| 200/200 [00:07<00:00, 27.42it/s]
2025-04-29  08:27:16 INFO Creating 200 train annotations (pascal) for class 'traffic light'
100%|██████████| 200/200 [00:00<00:00, 3387.76it/s]


{'traffic light': {'images_dir': './../data/traffic light/images',
  'annotations_dir': './../data/traffic light/pascal'}}

## 2 - Change data folder

In [4]:
import subprocess

subprocess.run(["mv", "../data/traffic light", "../data/traffic_light"])

CompletedProcess(args=['mv', '../data/traffic light', '../data/traffic_light'], returncode=0)

In [5]:
# Célula 1: Importações e Definições de Funções
import os
import shutil
from sklearn.model_selection import train_test_split
import random
from pathlib import Path

def create_directories(base_path):
    """Cria as pastas necessárias para treino e validação"""
    dirs = ['images/train', 'images/val', 'labels/train', 'labels/val']
    for dir_path in dirs:
        os.makedirs(os.path.join(base_path, dir_path), exist_ok=True)

def split_dataset(source_dir, output_dir, val_split=0.2, seed=42):
    """
    Separa o dataset em treino e validação
    
    Args:
        source_dir: Diretório com as imagens e labels originais
        output_dir: Diretório onde serão criadas as pastas train/val
        val_split: Proporção do dataset para validação (padrão: 0.2)
        seed: Seed para reproducibilidade
    """
    random.seed(seed)
    
    source_dir = Path(source_dir)
    output_dir = Path(output_dir)
    
    create_directories(output_dir)
    
    # Lista todos os arquivos de imagem
    image_files = [f for f in (source_dir / 'images').glob('*.jpg')]
    
    # Divide os arquivos em treino e validação
    train_files, val_files = train_test_split(image_files, 
                                            test_size=val_split,
                                            random_state=seed)
    
    def copy_files(files, split_type):
        for f in files:
            # Copia imagem
            dst_img = output_dir / 'images' / split_type / f.name
            shutil.copy2(f, dst_img)
            
            # Procura e copia label correspondente
            label_name = f.stem + '.xml'
            src_label = source_dir / 'pascal' / label_name
            if src_label.exists():
                dst_label = output_dir / 'labels' / split_type / label_name
                shutil.copy2(src_label, dst_label)
    
    # Copia arquivos para as respectivas pastas
    copy_files(train_files, 'train')
    copy_files(val_files, 'val')
    
    print(f'Dataset dividido com sucesso!')
    print(f'Treino: {len(train_files)} imagens')
    print(f'Validação: {len(val_files)} imagens')

In [6]:
# Célula 2: Execução da função
source_directory = "../data/traffic_light"  # diretório com as imagens originais
output_directory = "../datasets"               # diretório onde serão criadas as subpastas

split_dataset(
    source_dir=source_directory,
    output_dir=output_directory,
    val_split=0.2,
    seed=42
)

Dataset dividido com sucesso!
Treino: 160 imagens
Validação: 40 imagens


In [7]:
# Célula 3: Verificação
def verify_splits():
    base_dir = Path("../datasets")
    
    for split in ['train', 'val']:
        images = list((base_dir / 'images' / split).glob('*.jpg'))
        labels = list((base_dir / 'labels' / split).glob('*.txt'))
        
        print(f'Split {split}:')
        print(f'  Imagens: {len(images)}')
        print(f'  Labels: {len(labels)}')
        print(f'  Matched: {len(images) == len(labels)}')
        print()

verify_splits()

Split train:
  Imagens: 160
  Labels: 0
  Matched: False

Split val:
  Imagens: 40
  Labels: 0
  Matched: False



In [8]:
import os

train_image_names = os.listdir('../datasets/images/train')
train_image_names = [name.split('.')[0] for name in train_image_names]
train_image_names = set(train_image_names)

train_label_names = os.listdir('../datasets/labels/train')
train_label_names = [name.split('.')[0] for name in train_label_names]
train_label_names = set(train_label_names)

train_image_names == train_label_names

True

In [9]:
val_image_names = os.listdir('../datasets/images/val')
val_image_names = [name.split('.')[0] for name in val_image_names]
val_image_names = set(val_image_names)

val_label_names = os.listdir('../datasets/labels/val')
val_label_names = [name.split('.')[0] for name in val_label_names]
val_label_names = set(val_label_names)

val_image_names == val_label_names

True

In [10]:
import xml.etree.ElementTree as ET
import os
from pathlib import Path

def convert_pascal_to_yolo(xml_path, image_width, image_height):
    """
    Converte as coordenadas de Pascal VOC para formato YOLO

    Formato YOLO: <class> <x_center> <y_center> <width> <height>
    Onde todas as coordenadas são normalizadas entre 0 e 1
    """
    tree = ET.parse(xml_path)
    root = tree.getroot()

    yolo_annotations = []

    for obj in root.findall('object'):
        name = obj.find('name').text
        class_id = 0  # assumindo que só temos uma classe (traffic light)

        bbox = obj.find('bndbox')
        xmin = float(bbox.find('xmin').text)
        ymin = float(bbox.find('ymin').text)
        xmax = float(bbox.find('xmax').text)
        ymax = float(bbox.find('ymax').text)

        # Converter para formato YOLO (x_center, y_center, width, height)
        x_center = ((xmin + xmax) / 2) / image_width
        y_center = ((ymin + ymax) / 2) / image_height
        width = (xmax - xmin) / image_width
        height = (ymax - ymin) / image_height

        yolo_annotations.append(f"{class_id} {x_center:.6f} {y_center:.6f} {width:.6f} {height:.6f}")

    return yolo_annotations

def convert_dataset(base_path, label_dir_name='_labels'):
    """
    Converte todas as anotações do dataset
    """
    splits = ['train', 'val']

    for split in splits:
        label_dir = Path(base_path) / label_dir_name / split
        image_dir = Path(base_path) / 'images' / split

        # Criar diretório para as anotações YOLO se não existir
        yolo_label_dir = Path(base_path) / 'labels' / split
        yolo_label_dir.mkdir(parents=True, exist_ok=True)

        xml_files = list(label_dir.glob('*.xml'))

        for xml_file in xml_files:
            # Obter dimensões da imagem correspondente
            img_file = image_dir / f"{xml_file.stem}.jpg"
            if not img_file.exists():
                print(f"Imagem não encontrada para {xml_file}")
                continue

            from PIL import Image
            with Image.open(img_file) as img:
                img_width, img_height = img.size

            # Converter anotações
            yolo_annotations = convert_pascal_to_yolo(xml_file, img_width, img_height)

            # Salvar anotações no formato YOLO
            yolo_file = yolo_label_dir / f"{xml_file.stem}.txt"
            with open(yolo_file, 'w') as f:
                f.write('\n'.join(yolo_annotations))

        print(f"Convertidas {len(xml_files)} anotações para o split {split}")

In [11]:
import subprocess

subprocess.run(["mv", "../datasets/labels", "../datasets/_labels"])

CompletedProcess(args=['mv', '../datasets/labels', '../datasets/_labels'], returncode=0)

In [12]:

# Executar a conversão
convert_dataset('./../datasets', '_labels')

Convertidas 160 anotações para o split train
Convertidas 40 anotações para o split val


## Two Step Yolo

In [13]:
# Install YOLOv8 (Ultralytics)
!pip install ultralytics

# For CNN
import torch
import torchvision
import torchvision.transforms as transforms
import torch.nn as nn
import torch.optim as optim

# Other tools
import os
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt

# For loading the YOLO model
from ultralytics import YOLO
from IPython.display import clear_output
clear_output()

In [15]:
# Load a pre-trained YOLOv8n (nano) model
model = YOLO('yolov8n.pt')  # or yolov8s.pt for slightly bigger

import yaml

yaml_config = """
path: ./datasets/
train: images/train
val: images/val

nc: 1  # número de classes
names: ['traffic_light']  # nome das classes
"""

# Create a temporary YAML file
with open('dataset.yaml', 'w') as f:
    f.write(yaml_config)

# Train the model on your dataset
model.train(
    # inline yaml
    data='dataset.yaml',
    epochs=50,
    imgsz=320,
    batch=16,
    name='traffic_light_detector',
    save=True
)


New https://pypi.org/project/ultralytics/8.3.120 available 😃 Update with 'pip install -U ultralytics'
Ultralytics 8.3.119 🚀 Python-3.12.3 torch-2.7.0+cu126 CPU (12th Gen Intel Core(TM) i5-1235U)
engine/trainer: task=detect, mode=train, model=yolov8n.pt, data=dataset.yaml, epochs=50, time=None, patience=100, batch=16, imgsz=320, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=traffic_light_detector15, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=Fa

train: Scanning /home/erlo/Documents/Code/dl-projeto-final/src/datasets/labels/train... 160 images, 0 backgrounds, 0 corrupt: 100%|██████████| 160/160 [00:00<00:00, 1140.44it/s]

train: New cache created: /home/erlo/Documents/Code/dl-projeto-final/src/datasets/labels/train.cache
val: Fast image access ✅ (ping: 0.0±0.0 ms, read: 1002.1±233.6 MB/s, size: 434.7 KB)



val: Scanning /home/erlo/Documents/Code/dl-projeto-final/src/datasets/labels/val... 40 images, 0 backgrounds, 0 corrupt: 100%|██████████| 40/40 [00:00<00:00, 532.48it/s]

val: New cache created: /home/erlo/Documents/Code/dl-projeto-final/src/datasets/labels/val.cache


...

                   all         40        132      0.437      0.356      0.322      0.136



50 epochs completed in 0.235 hours.
Optimizer stripped from /home/erlo/Documents/Code/dl-projeto-final/runs/detect/traffic_light_detector15/weights/last.pt, 6.2MB
Optimizer stripped from /home/erlo/Documents/Code/dl-projeto-final/runs/detect/traffic_light_detector15/weights/best.pt, 6.2MB

Validating /home/erlo/Documents/Code/dl-projeto-final/runs/detect/traffic_light_detector15/weights/best.pt...
Ultralytics 8.3.119 🚀 Python-3.12.3 torch-2.7.0+cu126 CPU (12th Gen Intel Core(TM) i5-1235U)
Model summary (fused): 72 layers, 3,005,843 parameters, 0 gradients, 8.1 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 2/2 [00:01<00:00,  1.30it/s]


                   all         40        132      0.502      0.321      0.327      0.151
Speed: 0.5ms preprocess, 21.3ms inference, 0.0ms loss, 0.9ms postprocess per image
Results saved to /home/erlo/Documents/Code/dl-projeto-final/runs/detect/traffic_light_detector15


ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x76c36808cb60>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,     0.04004,    0.041041,    0.042042,    0.043043,    0.044044,    0.045045,    0.046046,    0.047047,
          0.048048, 

In [ ]:
import random
import cv2
import matplotlib.pyplot as plt
import numpy as np
import os
import torch

def load_random_image(image_dir):
    """Carrega uma imagem aleatória do diretório especificado"""
    image_files = [f for f in os.listdir(image_dir) if f.endswith(('.jpg', '.jpeg', '.png'))]
    random_image_name = random.choice(image_files)
    image_path = os.path.join(image_dir, random_image_name)
    return cv2.imread(image_path), random_image_name

def display_image_with_boxes(image, boxes, confidences, classes):
    """Exibe a imagem com as caixas delimitadoras"""
    image_display = image.copy()

    # Converte de volta para uint8 se necessário
    if image_display.dtype == np.float32:
        image_display = (image_display * 255).astype(np.uint8)

    for box, conf, cls in zip(boxes, confidences, classes):
        x1, y1, x2, y2 = map(int, box)
        cv2.rectangle(image_display, (x1, y1), (x2, y2), (0, 255, 0), 2)
        label = f'Class: {int(cls)}, Conf: {conf:.2f}'
        cv2.putText(image_display, label, (x1, y1-10),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

    plt.figure(figsize=(10, 10))
    plt.imshow(cv2.cvtColor(image_display, cv2.COLOR_BGR2RGB))
    plt.axis('off')
    plt.show()

# Carrega uma imagem aleatória do conjunto de treinamento
image_orig, image_name = load_random_image('../datasets/images/train')
image = cv2.resize(image_orig, (320, 320))  # Redimensiona para o tamanho de entrada do modelo

# Prepara a imagem para inferência
image_tensor = torch.from_numpy(image).float().permute(2, 0, 1).unsqueeze(0)
image_tensor = image_tensor / 255.0  # Normaliza para [0, 1]

# Executa a inferência
with torch.no_grad():
    results = model(image_tensor)

# Extrai as caixas, confianças e classes
boxes = results[0].boxes.xyxy.cpu().numpy()
confidences = results[0].boxes.conf.cpu().numpy()
classes = results[0].boxes.cls.cpu().numpy()

# Exibe a imagem com as detecções
display_image_with_boxes(image, boxes, confidences, classes)


0: 320x320 2 traffic_lights, 93.0ms
Speed: 0.0ms preprocess, 93.0ms inference, 1.7ms postprocess per image at shape (1, 3, 320, 320)


<Figure size 1000x1000 with 1 Axes>